In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

100%|██████████████████████████████████████████████████████████████| 170498071/170498071 [00:02<00:00, 64363248.15it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, criterion, epoch)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.301335
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.134032
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.773867
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.718294
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.540834
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.474840
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.355178
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.380234
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.189013
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.243093
Train Epoch: 3 [25600/50000 (51%)]	Loss: 1.431549
Train Epoch: 3 [38400/50000 (77%)]	Loss: 1.079669
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.385357
Train Epoch: 4 [12800/50000 (26%)]	Loss: 1.332109
Train Epoch: 4 [25600/50000 (51%)]	Loss: 1.236777
Train Epoch: 4 [38400/50000 (77%)]	Loss: 0.969942
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.193686
Train Epoch: 5 [12800/50000 (26%)]	Loss: 1.029075
Train Epoch: 5 [25600/50000 (51%)]	Loss: 0.967587
Train Epoch: 5 [38400/50000 (77%)]	Loss: 1.052698
Train Epoch: 6 [0/50000 (

In [9]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

test(model, device, test_loader, criterion)


Test set: Average loss: 0.0069, Accuracy: 7042/10000 (70%)



In [10]:
torch.save(model.state_dict(), "alexnet_cifar10.pth")
model = AlexNet(num_classes=10).to(device)
model.load_state_dict(torch.load("alexnet_cifar10.pth"))
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, 